In [1]:
from vizdoom import DoomGame  
import random
import time
import numpy as np
import gym
from gym import Env
from gym.spaces import Discrete, Box
import cv2
from matplotlib import pyplot as plt
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3 import PPO

In [2]:
import EnvironmentConfigurations as EnvConfig

In [3]:
from VizdoomGymWrapper import VizDoomGym

In [4]:
env = VizDoomGym(EnvConfig.configurations[EnvConfig.CURRENT_CONFIGURATION_INDEX], render=True)

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy
model = PPO.load(f"{EnvConfig.AGENT_MODEL_PATH_PREFIX}{EnvConfig.configurations[EnvConfig.CURRENT_CONFIGURATION_INDEX]['name']}/model_2975000")

FileNotFoundError: [Errno 2] No such file or directory: 'agents\\agent_for_deathmatch\\model_3000000.zip'

In [ ]:
from IPython.display import clear_output

for episode in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, second = model.predict(obs)
        obs, reward, done, info = env.step(action)
        # clear_output()
        # plt.imshow(obs)
        # plt.show()
        time.sleep(0.05555)
        total_reward += reward
        print(action, second)
    print("Total reward: ",total_reward)
    time.sleep(5)

2 None
2 None
3 None
2 None
2 None
2 None
2 None
2 None
3 None
3 None
3 None
2 None
3 None
2 None
2 None
2 None
3 None
3 None
3 None
3 None
3 None
2 None
3 None
2 None
3 None
3 None
3 None
2 None
2 None
3 None
3 None
3 None
3 None
3 None
2 None
3 None
3 None
3 None
1 None
1 None
3 None
2 None
3 None
2 None
2 None
2 None
2 None
3 None
3 None
1 None
2 None
2 None
2 None
2 None
1 None
2 None
1 None
2 None
2 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
2 None
3 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
2 None
3 None
2 None
3 None
3 None
3 None
3 None
2 None
3 None
1 None
1 None
2 None
3 None
3 None
3 None
2 None
3 None
3 None
2 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
3 None
1 None
1 None
1 None
1 None
2 None
1 None
2 None
1 None
2 None
1 None
2 None
1 None
2 None
1 None
2 None

ViZDoomUnexpectedExitException: Controlled ViZDoom instance exited unexpectedly.

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10)

In [ ]:
env.close()

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
model = PPO.load(f"{AGENT_MODEL_PATH_PREFIX}{configurations[CURRENT_CONFIGURATION_INDEX]['name']}/model_150000")
from IPython.display import clear_output

for episode in range(10):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        clear_output()
        plt.imshow(obs)
        plt.show()
        time.sleep(0.05)
    time.sleep(1)